# Climate Change Sentiment Analysis

### Import Packages

In [ ]:
import comet_ml 
from comet_ml import Experiment
# Create an experiment with your api key
experiment = Experiment(
    api_key=" ",
    project_name=" ",
    workspace=" ",
)

In [4]:
#reading data
import pandas as pd
import numpy as np
import string
import random
import re 

#visualization
from numpy import percentile
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set()
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#preprocessing and modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils import resample #used for downsampling
import sklearn.metrics as metrics
from sklearn.metrics import make_scorer
from collections import Counter 

#Natural Language Took Kit
import nltk
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
from tqdm import tqdm
import os
nltk.download('averaged_perceptron_tagger')
import spacy
import spacy
spacy.cli.download("en_core_web_sm")

from spacy.util import compounding
from spacy.util import minibatch

#For streamlit app
import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\climate.intern\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\climate.intern\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\climate.intern\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Load Data

In [6]:
train_df = pd.read_csv(r'C:\Users\climate.intern\OneDrive - MSF\Documents\GitHub\classification-predict-streamlit-template\train.csv')
test_df = pd.read_csv(r'C:\Users\climate.intern\OneDrive - MSF\Documents\GitHub\classification-predict-streamlit-template\test_with_no_labels.csv')

### Description of Data

In [8]:
print(train_df.shape)
print(test_df.shape)

display(train_df.head())
display(test_df.head())

(15819, 3)
(10546, 2)


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \r\nPu...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


The datasets aggregates tweets pertaining to climate change collected between Apr 27, 2015 and Feb 21, 2018.

The train data is structured as:

sentiment: Sentiment of tweet i.e 0, 1, etc
message: Tweet body
tweetid: Twitter unique id
The sentiments of the test data is the prediction target

#### Data Dimensions and Completeness

In [9]:
train_df['sentiment'].unique() 

array([ 1,  2,  0, -1], dtype=int64)

Sentiment Description:

2 News: the tweet links to factual news about climate change
1 Pro: the tweet supports the belief of man-made climate change
0 Neutral: the tweet neither supports nor refutes the belief of man-made climate change
-1 Anti: the tweet does not believe in man-made climate change

In [10]:
train_df["sentiment"].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

8530 tweets are believe, 3640 tweets are factual news about climate change, 2353 neither support nor refutes, and 1296 tweets do not believe in climate change

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15819 entries, 0 to 15818
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  15819 non-null  int64 
 1   message    15819 non-null  object
 2   tweetid    15819 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 370.9+ KB


##### Checking for missing values

In [12]:
train_df.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [13]:
test_df.isnull().sum()

message    0
tweetid    0
dtype: int64

Both the train and test datasets are complete, without null values.

In [14]:
# Looking for duplicates
percent_duplicates = round((1-(train_df['message'].nunique()/len(train_df['message'])))*100,2)
print('Duplicated tweets in train data:')
print(percent_duplicates,'%')

Duplicated tweets in train data:
10.05 %


## Exploratory Data Analysis